In [ ]:
#capture ste or others and then just bring in the suite numbers
#seems hard to get the after to not take in anything.
#\d{1,5} [\w\s]{1,20} (?:street|st|avenue|ave|road|rd|highway|hwy|square|sq|trail|trl|drive|dr|court|ct|park|parkway|pkwy|circle|cir|boulevard|blvd|lane|ln|way|spc|expressway|ste)(?:\s|\W)(?:\d{1,4})?(?:\s)?(?:(ne|nw|se|sw|n|s|e|w)\W)?(?:(ste\W|suite\W|space\W|box\W|bldg\W|#)(?:#\W|#)?[A-Za-z0-9]{0,6})?

In [83]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Matthew.DeLaney\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Matthew.DeLaney\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [1]:
import pandas as pd
import string
import math
import datetime as dt
import numpy as np
import re
import qgrid
from pivottablejs import pivot_ui
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.pipeline import make_pipeline
from sklearn import metrics


In [2]:
stopword = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()
lem = nltk.WordNetLemmatizer()

In [3]:
!jupyter nbextension enable --py --sys-prefix qgrid
!jupyter nbextension enable --py --sys-prefix widgetsnbextension

Enabling notebook extension qgrid/extension...
      - Validating: ok
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [4]:
social = pd.read_csv(r"C:\Users\matthew.delaney\OneDrive - Omnicom Media Group\Documents\ATTRework\socialcreative\full.csv",skiprows=1,encoding='UTF-8')

In [5]:
socialcopy =social.copy()

In [6]:
test = pd.read_csv(r"C:\Users\matthew.delaney\OneDrive - Omnicom Media Group\Documents\ATTRework\socialcreative\creativetextv2.csv",skiprows=1)

In [7]:
def tokenizer(s):
    tokens = re.split('\W+',s)
    return tokens

In [8]:
def remove_stopwords(tokenized):
    text = []
    for word in tokenized:
        if word not in stopword:
            text.append(word)
    return text
            

In [9]:
def stemming(tokenized):
    text = []
    for word in tokenized:
        text.append(ps.stem(word))
    return text

In [10]:
def lemmatizing(tokenized):
    text = []
    for word in tokenized:
        text.append(lem.lemmatize(word))
    return text    
    

In [11]:
def removal(x):

    #Punctionation
    remove = string.punctuation
    remove = remove.replace("#", "").replace('$','').replace('%','') # don't remove octothorpe |"$"|"%"
    pattern = r"[{}]".format(remove) # create the pattern
    punc = re.compile(pattern)
    emoji = re.compile( "["
    "\U0001F1E0-\U0001F1FF"  # flags (iOS)
    "\U0001F300-\U0001F5FF"  # symbols & pictographs
    "\U0001F600-\U0001F64F"  # emoticons
    "\U0001F680-\U0001F6FF"  # transport & map symbols
    "\U0001F700-\U0001F77F"  # alchemical symbols
    "\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
    "\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
    "\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
    "\U0001FA00-\U0001FA6F"  # Chess Symbols
    "\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
    "\U00002702-\U000027B0"  # Dingbats
    "\U000024C2-\U0001F251" 
    "]+")
    
    #websites
    link = re.compile('(?i)((?:https?://|www\d{0,3}[.])?[a-z0-9.\-]+[.](?:(?:international)|(?:construction)|(?:contractors)|(?:enterprises)|(?:photography)|(?:immobilien)|(?:management)|(?:technology)|(?:directory)|(?:education)|(?:equipment)|(?:institute)|(?:marketing)|(?:solutions)|(?:builders)|(?:clothing)|(?:computer)|(?:democrat)|(?:diamonds)|(?:graphics)|(?:holdings)|(?:lighting)|(?:plumbing)|(?:training)|(?:ventures)|(?:academy)|(?:careers)|(?:company)|(?:domains)|(?:florist)|(?:gallery)|(?:guitars)|(?:holiday)|(?:kitchen)|(?:recipes)|(?:shiksha)|(?:singles)|(?:support)|(?:systems)|(?:agency)|(?:berlin)|(?:camera)|(?:center)|(?:coffee)|(?:estate)|(?:kaufen)|(?:luxury)|(?:monash)|(?:museum)|(?:photos)|(?:repair)|(?:social)|(?:tattoo)|(?:travel)|(?:viajes)|(?:voyage)|(?:build)|(?:cheap)|(?:codes)|(?:dance)|(?:email)|(?:glass)|(?:house)|(?:ninja)|(?:photo)|(?:shoes)|(?:solar)|(?:today)|(?:aero)|(?:arpa)|(?:asia)|(?:bike)|(?:buzz)|(?:camp)|(?:club)|(?:coop)|(?:farm)|(?:gift)|(?:guru)|(?:info)|(?:jobs)|(?:kiwi)|(?:land)|(?:limo)|(?:link)|(?:menu)|(?:mobi)|(?:moda)|(?:name)|(?:pics)|(?:pink)|(?:post)|(?:rich)|(?:ruhr)|(?:sexy)|(?:tips)|(?:wang)|(?:wien)|(?:zone)|(?:biz)|(?:cab)|(?:cat)|(?:ceo)|(?:com)|(?:edu)|(?:gov)|(?:int)|(?:mil)|(?:net)|(?:onl)|(?:org)|(?:pro)|(?:red)|(?:tel)|(?:uno)|(?:xxx)|(?:ac)|(?:ad)|(?:ae)|(?:af)|(?:ag)|(?:ai)|(?:al)|(?:am)|(?:an)|(?:ao)|(?:aq)|(?:ar)|(?:as)|(?:at)|(?:au)|(?:aw)|(?:ax)|(?:az)|(?:ba)|(?:bb)|(?:bd)|(?:be)|(?:bf)|(?:bg)|(?:bh)|(?:bi)|(?:bj)|(?:bm)|(?:bn)|(?:bo)|(?:br)|(?:bs)|(?:bt)|(?:bv)|(?:bw)|(?:by)|(?:bz)|(?:ca)|(?:cc)|(?:cd)|(?:cf)|(?:cg)|(?:ch)|(?:ci)|(?:ck)|(?:cl)|(?:cm)|(?:cn)|(?:co)|(?:cr)|(?:cu)|(?:cv)|(?:cw)|(?:cx)|(?:cy)|(?:cz)|(?:de)|(?:dj)|(?:dk)|(?:dm)|(?:do)|(?:dz)|(?:ec)|(?:ee)|(?:eg)|(?:er)|(?:es)|(?:et)|(?:eu)|(?:fi)|(?:fj)|(?:fk)|(?:fm)|(?:fo)|(?:fr)|(?:ga)|(?:gb)|(?:gd)|(?:ge)|(?:gf)|(?:gg)|(?:gh)|(?:gi)|(?:gl)|(?:gm)|(?:gn)|(?:gp)|(?:gq)|(?:gr)|(?:gs)|(?:gt)|(?:gu)|(?:gw)|(?:gy)|(?:hk)|(?:hm)|(?:hn)|(?:hr)|(?:ht)|(?:hu)|(?:id)|(?:ie)|(?:il)|(?:im)|(?:in)|(?:io)|(?:iq)|(?:ir)|(?:is)|(?:it)|(?:je)|(?:jm)|(?:jo)|(?:jp)|(?:ke)|(?:kg)|(?:kh)|(?:ki)|(?:km)|(?:kn)|(?:kp)|(?:kr)|(?:kw)|(?:ky)|(?:kz)|(?:la)|(?:lb)|(?:lc)|(?:li)|(?:lk)|(?:lr)|(?:ls)|(?:lt)|(?:lu)|(?:lv)|(?:ly)|(?:ma)|(?:mc)|(?:md)|(?:me)|(?:mg)|(?:mh)|(?:mk)|(?:ml)|(?:mm)|(?:mn)|(?:mo)|(?:mp)|(?:mq)|(?:mr)|(?:ms)|(?:mt)|(?:mu)|(?:mv)|(?:mw)|(?:mx)|(?:my)|(?:mz)|(?:na)|(?:nc)|(?:ne)|(?:nf)|(?:ng)|(?:ni)|(?:nl)|(?:no)|(?:np)|(?:nr)|(?:nu)|(?:nz)|(?:om)|(?:pa)|(?:pe)|(?:pf)|(?:pg)|(?:ph)|(?:pk)|(?:pl)|(?:pm)|(?:pn)|(?:pr)|(?:ps)|(?:pt)|(?:pw)|(?:py)|(?:qa)|(?:re)|(?:ro)|(?:rs)|(?:ru)|(?:rw)|(?:sa)|(?:sb)|(?:sc)|(?:sd)|(?:se)|(?:sg)|(?:sh)|(?:si)|(?:sj)|(?:sk)|(?:sl)|(?:sm)|(?:sn)|(?:so)|(?:sr)|(?:st)|(?:su)|(?:sv)|(?:sx)|(?:sy)|(?:sz)|(?:tc)|(?:td)|(?:tf)|(?:tg)|(?:th)|(?:tj)|(?:tk)|(?:tl)|(?:tm)|(?:tn)|(?:to)|(?:tp)|(?:tr)|(?:tt)|(?:tv)|(?:tw)|(?:tz)|(?:ua)|(?:ug)|(?:uk)|(?:us)|(?:uy)|(?:uz)|(?:va)|(?:vc)|(?:ve)|(?:vg)|(?:vi)|(?:vn)|(?:vu)|(?:wf)|(?:ws)|(?:ye)|(?:yt)|(?:za)|(?:zm)|(?:zw))(?:/[^\s()<>]+[^\s`!()\[\]{};:\'".,<>?\xab\xbb\u201c\u201d\u2018\u2019])?)', re.IGNORECASE)
    websites = re.compile('^(?#Protocol)(?:(?:ht|f)tp(?:s?)\:\/\/|~\/|\/)?(?#Username:Password)(?:\w+:\w+@)?(?#Subdomains)(?:(?:[-\w]+\.)+(?#TopLevel Domains)(?:com|org|net|gov|mil|biz|info|mobi|name|aero|jobs|museum|travel|[a-z]{2}))(?#Port)(?::[\d]{1,5})?(?#Directories)(?:(?:(?:\/(?:[-\w~!$+|.,=]|%[a-f\d]{2})+)+|\/)+|\?|#)?(?#Query)(?:(?:\?(?:[-\w~!$+|.,*:]|%[a-f\d{2}])+=?(?:[-\w~!$+|.,*:=]|%[a-f\d]{2})*)(?:&(?:[-\w~!$+|.,*:]|%[a-f\d{2}])+=?(?:[-\w~!$+|.,*:=]|%[a-f\d]{2})*)*)*(?#Anchor)(?:#(?:[-\w~!$+|.,*:=]|%[a-f\d]{2})*)?$',re.IGNORECASE)
    https = re.compile('http\S+', re.IGNORECASE)

    #date and time
    date = re.compile('(?:(?<!\:)(?<!\:\d)[0-3]?\d(?:st|nd|rd|th)?\s+(?:of\s+)?(?:jan\.?|january|feb\.?|february|mar\.?|march|apr\.?|april|may|jun\.?|june|jul\.?|july|aug\.?|august|sep\.?|september|oct\.?|october|nov\.?|november|dec\.?|december)|(?:jan\.?|january|feb\.?|february|mar\.?|march|apr\.?|april|may|jun\.?|june|jul\.?|july|aug\.?|august|sep\.?|september|oct\.?|october|nov\.?|november|dec\.?|december)\s+(?<!\:)(?<!\:\d)[0-3]?\d(?:st|nd|rd|th)?)(?:\,)?\s*(?:\d{4})?|[0-3]?\d[-\./][0-3]?\d[-\./]\d{2,4}', re.IGNORECASE)
    time = re.compile('\d{1,2}:\d{2} ?(?:[ap]\.?m\.?)?(?: [ECMP]T)?|\d[ap]\.?m\.?(?: [ECMP]T)?', re.IGNORECASE)
    dateMDY = re.compile('(1[0-2]|[1-9])\/(3[01]|[12][0-9]|[1-9])\/(\d{2,4})')
    dateMD = re.compile('(1[0-2]|[1-9])\/(3[01]|[12][0-9]|[1-9])')
    
    #phones
    phone = re.compile('''((?:(?<![\d-])(?:\+?\d{1,3}[-.\s*]?)?(?:\(?\d{3}\)?[-.\s*]?)?\d{3}[-.\s*]?\d{4}(?![\d-]))|(?:(?<![\d-])(?:(?:\(\+?\d{2}\))|(?:\+?\d{2}))\s*\d{2}\s*\d{3}\s*\d{4}(?![\d-])))''')
    phones_with_exts = re.compile('((?:(?:\+?1\s*(?:[.-]\s*)?)?(?:\(\s*(?:[2-9]1[02-9]|[2-9][02-8]1|[2-9][02-8][02-9])\s*\)|(?:[2-9]1[02-9]|[2-9][02-8]1|[2-9][02-8][02-9]))\s*(?:[.-]\s*)?)?(?:[2-9]1[02-9]|[2-9][02-9]1|[2-9][02-9]{2})\s*(?:[.-]\s*)?(?:[0-9]{4})(?:\s*(?:#|x\.?|ext\.?|extension)\s*(?:\d+)?))', re.IGNORECASE)
    
    #curr = r"(?:[\£\$\€]{1}[,\d]+.?\d*)"


    #address = re.compile('\d{1,5} [\w\s]{1,20}(?:street|st|avenue|ave|road|rd|highway|hwy|square|sq|trail|trl|drive|dr|court|ct|park|parkway|pkwy|circle|cir|boulevard|blvd|lane|ln)\W?(?=\s|$)', re.IGNORECASE)
    address = re.compile('\d{1,5} [\w\s]{1,20} (?:street|st|avenue|ave|road|rd|highway|hwy|square|sq|trail|trl|drive|dr|court|ct|park|parkway|pkwy|circle|cir|boulevard|blvd|lane|ln|way|spc|expressway|pike|fwy|ctr|pl|route)(?:\s|\W)(?:\d{1,4})?(?:\s)?(?:(ne|nw|se|sw|n|s|e|w)\W)?(?:(ste\W|suite\W|space\W|box\W|bldg\W|unit\W|#)(?:#\W|#)?[A-Za-z0-9]{0,6})?',re.IGNORECASE)
    suite   = re.compile('\b(?:suite|ste)(?: # \S{1,6}| \S{1,6})')
    #characters = r'[\W\_]'
    
    x = link.sub(' ',str(x))
    x = date.sub(' ',str(x))
    x = time.sub(' ',str(x))
    x = websites.sub(' ',str(x))
    x = https.sub(' ',str(x))
    
    x = address.sub(' ',str(x))
    
    x = dateMDY.sub(' ',str(x))
    x = dateMD.sub(' ',str(x))
    
    x = phone.sub(' ',str(x))
    x = phones_with_exts.sub(' ',str(x)) 
    
    x = punc.sub(' ',str(x))
    x = emoji.sub('',str(x))
    #x = re.sub(r'[^\w\s]',' ',str(x))
    return x

In [12]:
test['creative_text_clean'] = test['Creative Text'].apply(lambda x: removal(x))
#test['creative_text_token'] = test['creative_text_clean'].apply(lambda x: tokenizer(x.lower()))
#test['creative_text_no_stop'] = test['creative_text_token'].apply(lambda x: remove_stopwords(x))
#test['creative_test_stemmed'] = test['creative_text_no_stop'].apply(lambda x: stemming(x))
#test['creative_test_lemmatized'] = test['creative_text_no_stop'].apply(lambda x: lemmatizing(x))

In [13]:
sample = test.sample(frac=.02, random_state=82)

In [14]:
sample.to_excel(r"C:\Users\matthew.delaney\OneDrive - Omnicom Media Group\Documents\ATTRework\socialcreative\sample.xlsx",index=False)

In [15]:
def tfidftokenizer(s):
    text = []
    tokens = re.split('\W+',s)
    for word in tokens:
        if word not in stopword:
            text.append(lem.lemmatize(word))
    return text

In [24]:
labeled = pd.read_excel(r"C:\Users\matthew.delaney\OneDrive - Omnicom Media Group\Documents\ATTRework\socialcreative\sampleMD.xlsx")

In [25]:
labeled['creative_text_clean'] = labeled['Creative Text'].apply(lambda x: removal(x))

In [23]:
list(sample)

['Advertiser',
 'Brand Root',
 'Brand (Major)',
 'Brand (Minor)',
 'Brand (Leaf)',
 'Link to Creative',
 'Creative Text',
 'creative_text_clean']

In [26]:
combined = pd.merge(labeled,socialcopy,how="left",on=['Creative Text','Advertiser'])

In [27]:
dfcombined = combined.drop(['Brand (Minor)', 'Brand (Leaf)','Category','Region','Type','Landing Page',
 'Link to Creative', 'Creative ID'],axis=1,inplace=False)

In [28]:
dfcombined['LOB'] = dfcombined['LOB'].str.lower()

In [29]:
rt = ['Advertiser',
 'LOB',
 'Site',
 'Device',
 'Brand Root',
 'Brand (Major)']

for x  in rt:
    print(x)
    print(dfcombined[x].unique())
    print(len(dfcombined[x].unique()))
    #print(dfcombined[x].isnull().values.any())

#dfcombined['Advertiser'].unique

Advertiser
['Cox Communications, Inc.' 'Spectrum (Charter Communications)' 'Sprint'
 'CenturyLink, Inc.' 'AT&T' 'Verizon' 'Comcast Corporation' 'DirecTV'
 'Dish Network' 'CSC Holdings, LLC.' 'Altice N.V.'
 'Time Warner Cable Enterprises LLC' 'Netflix, Inc.' 'Amazon.com'
 'Bright House Networks' 'T-Mobile' 'CBS (cbs.com)' 'Sling TV (by Dish)'
 'NBC Universal Television (nbc.com)' 'Hulu' 'Apple Inc.' 'Disney'
 'Home Box Office, Inc. (HBO)' 'Boost Mobile (Boost Worldwide, Inc.)'
 'Cricket Communications, Inc.' 'TracFone Wireless, Inc.' 'Lycamobile'
 'MetroPCS Wireless, Inc.' 'Mint Mobile (Ultra Mobile (UVNV, Inc.))'
 'Consumer Cellular' 'Virgin Mobile']
31
LOB
['b2b' 'bbv' 'corp' 'other' 'ott' 'post' 'pre']
7
Site
['facebook.com' 'twitter.com']
2
Device
['Facebook' 'Twitter']
2
Brand Root
['Business Solutions' 'All Business Services' 'Corporate' '<unbranded>'
 'All B2B Non-Wireless Solutions' 'All Consumer Services' 'DirecTV Now'
 'All Consumer In-Home Services' 'Optimum' 'Suddenlink Comm

In [30]:
le = preprocessing.LabelEncoder()
dfcombined['LOB_vector'] = le.fit_transform(dfcombined['LOB'])

In [33]:
from sklearn.model_selection import train_test_split
labels = dfcombined['LOB_vector'].copy()
features = dfcombined[['Advertiser','Brand (Major)','creative_text_clean']].copy()

x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size = 0.2, random_state=0)


In [34]:
MAX_FEAT_DESCP = 50000


preprocess = ColumnTransformer(
    [('advertiser_category', OneHotEncoder(dtype='int', handle_unknown='ignore'), ['Advertiser']),
     ('brand_major_cat', OneHotEncoder(dtype='int', handle_unknown='ignore'), ['Brand (Major)']),
     ('creativeclean_tfidf', TfidfVectorizer(max_features = MAX_FEAT_DESCP, stop_words = 'english', ngram_range=(1,3)), 'creative_text_clean')])

In [35]:
model = make_pipeline(preprocess,RandomForestClassifier(n_estimators=100))

In [36]:
model.fit(x_train,y_train)

Pipeline(memory=None,
         steps=[('columntransformer',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('advertiser_category',
                                                  OneHotEncoder(categorical_features=None,
                                                                categories=None,
                                                                drop=None,
                                                                dtype='int',
                                                                handle_unknown='ignore',
                                                                n_values=None,
                                                                sparse=True),
                                                  ['Advertiser']),
                                              

In [37]:
y_pred = model.predict(x_test)

In [42]:
from sklearn.metrics import accuracy_score

In [43]:
accuracy_score(y_test,y_pred)

0.9904214559386973

In [44]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model, features, labels, cv=10)
print(scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

[0.95238095 0.93142857 0.92938931 0.91395793 0.99042146 0.97509579
 0.89443378 0.92692308 0.87861272 0.89017341]
Accuracy: 0.93 (+/- 0.07)


In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

erf_clf = ExtraTreesClassifier(n_estimators=10, max_depth=None, min_samples_split=1, random_state=0)


scores = sklearn.cross_validation.cross_val_score(erf_clf, features, labels, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
